In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import missingno as msno
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score
from pandas.plotting import scatter_matrix

In [ ]:
#Read in datasets
df=pd.read_sas("/Users/alexmaillis/Downloads/P_DEMO.xpt")
huq=pd.read_sas("/Users/alexmaillis/Downloads/P_HUQ.xpt")
bmi=pd.read_sas("/Users/alexmaillis/Downloads/P_BMX.xpt")
diabetes=pd.read_sas("/Users/alexmaillis/Downloads/P_DIQ.xpt")
fitness=pd.read_sas("/Users/alexmaillis/Downloads/P_PAQ.xpt")
glu=pd.read_sas("/Users/alexmaillis/Downloads/P_GLU.xpt")
#checking that data imported 
glu.head(5)

In [ ]:
#1) renaming columns 
df=df.rename(columns={"SEQN":"ID","RIAGENDR":"gender","RIDAGEYR":"age_yrs","DMDMARTZ":"married_cat","DMDEDUC2":"edu_cat"})
huq=huq.rename(columns={"SEQN":"ID","HUQ010":"health_cond","HUQ051":"visits_per_yr","HUQ071":"IP_within_yr"})
bmi=bmi.rename(columns={"SEQN":"ID"})
fitness=fitness.rename(columns={"SEQN":"ID","PAD680":"min_sedentary","PAQ640":"days_walk"})
glu=glu.rename(columns={"SEQN":"ID","LBXGLU":"fasting_glu"})
df_y=diabetes.rename(columns={"SEQN":"ID","DIQ010":"y0_dm_doctor","DIQ050":"y1_dm_insulin","DIQ280":"y2_dm_a1c"})
#2) convert variables in scientific notation to integers
pd.set_option('display.float_format', '{:.2f}'.format)
# 3) check  
huq.head(10)

In [ ]:
#Join datasets by ID 
df_1=df.set_index('ID').join(huq.set_index('ID'), on="ID",how="left").join(df_y.set_index('ID'), on="ID",how="left").join(bmi.set_index('ID'),how="left").join(fitness.set_index('ID'),how="left").\
    join(glu.set_index('ID'), on="ID",how="left")
df_1.reset_index(inplace=True)
df_2 = df_1[['ID', 'gender','age_yrs','married_cat','edu_cat','BMXBMI','BMDBMIC','fasting_glu','min_sedentary','days_walk','health_cond','visits_per_yr','IP_within_yr'
            , 'y0_dm_doctor','y1_dm_insulin','y2_dm_a1c']]
df_2.head(10)

In [ ]:
#THE NEXT 3 CELLS CHECK DATA DISTRIBUTION AND MISSINGNESS

In [ ]:
print(df_2.describe())
print(df_2.info())

In [ ]:
print(pd.crosstab(df_2['age_yrs'], df_2['BMDBMIC']))
#BMI Category is only available for age_yrs < 20. will use numeric value in model 

In [ ]:
#%matplotlib inline
#df_2.hist(bins=60, figsize=(20,15))
#plt.show()

data=df_2.visits_per_yr
data1=df_2.BMXBMI
data2=df_2.min_sedentary
#print(plt.hist(data, bins=60,color='red',edgecolor='black'))
#print(plt.hist(data1, bins=60,color='blue',edgecolor='black'))
print(plt.hist(data2, bins=100,color='green',edgecolor='black'))

In [ ]:
print(msno.matrix(df_2))
#print(msno.bar(df_2))
print(msno.heatmap(df_2))

In [ ]:
#Clean Outcome variable and remove 'unknown'/'refused' observations
# Filter out rows with incomplete visit # and min_sedentary data
remove=[99.00]
remove1=[9999.0,7777.0]
df_3 = df_2[~df_2['visits_per_yr'].isin(remove)].copy()
df_4 = df_3[~df_3['min_sedentary'].isin(remove1)].copy()
df_5 = df_4[df_4['age_yrs'] > 17].copy()
#recode diabetes to 0/1
df_5['y0_dm_bin'] = df_5['y0_dm_doctor'].apply(lambda x: 1 if x == 1.00 else 0)

#check binary variable accuracy
print(pd.crosstab(df_5['y0_dm_bin'], df_5['y0_dm_doctor']))

df_6 = df_5[['gender','age_yrs','BMXBMI','min_sedentary','health_cond','visits_per_yr','IP_within_yr','y0_dm_bin']]
df_6 = df_6.dropna()
print(df_6.describe())

In [ ]:
#Check: stratified sampling vs random sampling? per CDC, 40% of americans are obese. Is my sample similar? 
print(len(df_6[df_6['BMXBMI'] >= 30.00]))
print(3647/8701)
#42% of data being included in model has BMI>=30.0 (obese). Will stick with random sampling vs. stratified

total_missing = df_6.isnull().sum().sum()

print(f"Total missing values in the DataFrame: {total_missing}")

In [ ]:
#split data for testing
train_set, test_set = train_test_split(df_6,test_size=0.2,random_state=42)

#Check for multicollinearity in training data
print(train_set.corr())

#Split predictors from labels
dm_train=train_set.drop('y0_dm_bin',axis=1)
dm_labels=train_set['y0_dm_bin'].copy()

In [ ]:
#Numeric and Categorical transformation pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import StandardScaler, OneHotEncoder

class DataFrameSelector(BaseEstimator,TransformerMixin):
    def __init__(self,attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values
        
dm_num=dm_train[['age_yrs','BMXBMI','min_sedentary']]
dm_cat=dm_train[['gender','health_cond','visits_per_yr','IP_within_yr']]
num_attributes = list(dm_num)
cat_attributes = list(dm_cat)

num_pipeline=Pipeline([
    ('selector',DataFrameSelector(num_attributes)),
    ('std_scaler',StandardScaler()),
])

cat_pipeline = Pipeline([
    ('selector',DataFrameSelector(cat_attributes)),
    ('cat_encoder', OneHotEncoder(handle_unknown='ignore')),
])

full_pipeline = FeatureUnion(transformer_list=[
    ('num_pipeline', num_pipeline),
    ('cat_pipeline', cat_pipeline),
])

dm_processed=full_pipeline.fit_transform(dm_train)
dm_processed

In [ ]:
#TRAIN LOGISTIC MODEL 

log_reg=LogisticRegression()
log_reg.fit(dm_processed,dm_labels)
print("model score: %.3f" % log_reg.score(dm_processed,dm_labels))

In [ ]:
#Now what? 